In [ ]:

import tensorflow as tf
import datetime
from tensorflow import keras
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, BatchNormalization
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras import backend as K
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

#with tf.device('/gpu:0'): 
#    sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True))

NUM_EPOCHS = 100
INIT_LR = 1e-3
BATCH_SIZE = 50

cifar100_mnist = tf.keras.datasets.cifar100

(train_images, train_labels), (test_images, test_labels) = cifar100_mnist.load_data()

train_images = train_images / 255.0
test_images = test_images / 255.0

droprate = 0.2
verbosity = 1
num_classes =100
input_shape = (32,32,3)

if K.image_data_format() == "channels_first":
    train_images_cnn = train_images.reshape((train_images.shape[0], 3, 32, 32))
    test_images_cnn = test_images.reshape((test_images.shape[0], 3, 32, 32))
else:
    train_images_cnn = train_images.reshape((train_images.shape[0], 32, 32, 3))
    test_images_cnn = test_images.reshape((test_images.shape[0], 32, 32, 3))

img_input = tf.keras.Input(shape=(input_shape))


    
class MyModel(tf.keras.Model):
    def __init__(self, num_classes=100):
        super(MyModel,self).__init__(name='my_model')
        self.num_classes=num_classes
        self.conv1 = tf.keras.layers.Conv2D(32, 3, activation='relu')
        self.conv2 = tf.keras.layers.Conv2D(64, 3, activation='relu')
        self.batch = tf.keras.layers.BatchNormalization()
        self.dropout = tf.keras.layers.Dropout(0.5)
        self.flatten1 = tf.keras.layers.Flatten()
        self.dense_1 = tf.keras.layers.Dense(1000,activation='relu')
        self.dense_2 = tf.keras.layers.Dense(1000,activation='relu')
        self.dense_3 = tf.keras.layers.Dense(1000,activation='relu')
        self.dense_4 = tf.keras.layers.Dense(num_classes,activation='softmax')
        
    def call(self,inputs):
        a=self.conv1(inputs)
        b=self.conv2(a)
        
        c=self.flatten1(b)
        
        d=self.dense_1(c)
        e=self.dropout(d)
        f=self.batch(e)
        
        g = self.dense_2(f)
        h=self.dropout(g)
        i=self.batch(h)
        
        j = self.dense_3(i)
        k=self.dropout(j)
        l=self.batch(k)
        
        return self.dense_4(l)
    
    def compute_output_shape(self,input_shape):
        shape=tf.TensorShape(input_shape).as_list()
        shape[-1]=self.num_classes
        return tf.TensorShape(shape)

model = MyModel(100)

opt = tf.optimizers.Adam(learning_rate=INIT_LR)
opt2 = tf.optimizers.SGD(lr=INIT_LR, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(optimizer= opt,loss='sparse_categorical_crossentropy',metrics=['accuracy'])

call_back = [tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss')]

H=model.fit(train_images_cnn, train_labels,epochs=NUM_EPOCHS,callbacks=call_back)

test_loss, test_acc = model.evaluate(test_images_cnn, test_labels)

probability_model = tf.keras.Sequential([model,tf.keras.layers.Softmax()])
print('Test accuracy:', test_acc)
predictions = model.predict(test_images_cnn)

# Visualize history
# Plot history: Loss
plt.plot(H.history['val_loss'])
plt.title('Validation loss history')
plt.ylabel('Loss value')
plt.xlabel('No. epoch')
plt.show()

# Plot history: Accuracy
plt.plot(H.history['val_accuracy'])
plt.title('Validation accuracy history')
plt.ylabel('Accuracy value (%)')
plt.xlabel('No. epoch')
plt.show()


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, BatchNormalization
from tensorflow.keras.layers import MaxPooling2D, Dropout
from tensorflow.keras import backend as K
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import os
#이미지 불러오기
cifar100 = tf.keras.datasets.cifar100
(train_images, train_labels), (test_images, test_labels) = cifar100.load_data()
classes_name = [
    'apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle',
    'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel',
    'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock',
    'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur',
    'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster',
    'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion',
    'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse',
    'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear',
    'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine',
    'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose',
    'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake',
    'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table',
    'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout',
    'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman',
    'worm'
]
# nomalization
train_images = train_images / 255.0
test_images = test_images / 255.0

num_classes = 100
NUM_EPOCHS =100

model = Sequential()

model.add(Conv2D(32,kernel_size=(3,3),activation='relu', kernel_initializer='he_uniform', input_shape=(32,32,3)))
#he_uniform : 균일 분산 스케일링
model.add(Conv2D(32,kernel_size=(3,3),kernel_initializer='he_uniform',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(1000, activation='relu'))

model.add(Dense(100, activation='softmax'))


if K.image_data_format() == "channels_first":
    train_images_cnn = train_images.reshape((train_images.shape[0], 3, 32, 32))
    test_images_cnn = test_images.reshape((test_images.shape[0], 3, 32, 32))
# otherwise, we are using "channels last" ordering, so the design
# matrix shape should be: num_samples x rows x columns x depth
else:
    train_images_cnn = train_images.reshape((train_images.shape[0], 32, 32, 3))
    test_images_cnn = test_images.reshape((test_images.shape[0], 32, 32, 3))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])



log_dir =  datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback=tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1)

b = os.getcwd()
print(b)

H = model.fit(train_images_cnn, train_labels, epochs=NUM_EPOCHS,callbacks=[tensorboard_callback])


#정확도
test_loss, test_acc = model.evaluate(test_images_cnn, test_labels)
print('Test accuracy:', test_acc)
#모델 예측
predictions = model.predict(test_images_cnn)
#그래프로 출력하기
N = NUM_EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
#plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss") # no validation data for this example
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
#plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc") # no validation data for this example
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.show()

C:\Users\fopguy0908\Python_Programming
Train on 50000 samples
Epoch 1/100
50000/50000 [==============================] - 6s 127us/sample - loss: 3.2809 - accuracy: 0.2399
Epoch 2/100
50000/50000 [==============================] - 6s 111us/sample - loss: 2.2905 - accuracy: 0.4116
Epoch 3/100
50000/50000 [==============================] - 6s 112us/sample - loss: 1.7756 - accuracy: 0.5236
Epoch 4/100
50000/50000 [==============================] - 6s 110us/sample - loss: 1.3321 - accuracy: 0.6242
Epoch 5/100
50000/50000 [==============================] - 6s 112us/sample - loss: 0.9435 - accuracy: 0.7256
Epoch 6/100
50000/50000 [==============================] - 6s 113us/sample - loss: 0.6928 - accuracy: 0.7896
Epoch 7/100
50000/50000 [==============================] - 6s 110us/sample - loss: 0.5528 - accuracy: 0.8291
Epoch 8/100
50000/50000 [==============================] - 6s 111us/sample - loss: 0.4545 - accuracy: 0.8585
Epoch 9/100
50000/50000 [==============================] - 6s 111u

Epoch 75/100
50000/50000 [==============================] - 6s 112us/sample - loss: 0.0584 - accuracy: 0.9819
Epoch 76/100
50000/50000 [==============================] - 6s 114us/sample - loss: 0.0595 - accuracy: 0.9827
Epoch 77/100
50000/50000 [==============================] - 6s 115us/sample - loss: 0.0545 - accuracy: 0.9833
Epoch 78/100
50000/50000 [==============================] - 6s 114us/sample - loss: 0.0562 - accuracy: 0.9821
Epoch 79/100
50000/50000 [==============================] - 6s 112us/sample - loss: 0.0570 - accuracy: 0.9825
Epoch 80/100
50000/50000 [==============================] - 6s 112us/sample - loss: 0.0546 - accuracy: 0.9833
Epoch 81/100
50000/50000 [==============================] - 6s 112us/sample - loss: 0.0517 - accuracy: 0.9839
Epoch 82/100
50000/50000 [==============================] - 6s 115us/sample - loss: 0.0505 - accuracy: 0.9843
Epoch 83/100
50000/50000 [==============================] - 6s 114us/sample - loss: 0.0533 - accuracy: 0.9843
Epoch 84/1